In [2]:
# Import required libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


In [3]:
# Load dataset
df=pd.read_csv('https://raw.githubusercontent.com/mohittomar2008/Flight-Price-Prediction-thru-Webscrapping/main/flight.csv', index_col=[0], parse_dates=['Booking_date','Journey_date'])

In [4]:
# Dataset random sample
df.sample(5)

,From,To,Booking_date,Journey_date,Airline,Code,Departure,Arrival,Duration,Stops,Price
4702,Bangalore,Delhi,2022-06-24,2022-06-30,Vistara,UK-846,08:30,14:30,06h 00m,2+-stop,"15,872"
6176,Kolkata,Bangalore,2022-06-24,2022-06-26,Vistara,UK-774,20:35,19:15,22h 40m,1-stop,"12,046"
3878,Bangalore,Delhi,2022-06-24,2022-06-26,GO FIRST,G8- 116,05:45,08:40,02h 55m,non-stop,"9,679"
6899,Bangalore,kolkata,2022-06-24,2022-06-26,AirAsia,I5-992,23:30,02:05,02h 35m,non-stop,"7,789"
6052,Kolkata,Bangalore,2022-06-24,2022-06-25,Vistara,UK-776,17:35,23:10,05h 35m,1-stop,"12,046"


In [5]:
df.shape

(7548, 11)

In [6]:
df.isnull().sum() # target feature have some Null values, so better to ignore those records

From               0
To                 0
Booking_date       0
Journey_date       0
Airline            0
Code               0
Departure          0
Arrival            0
Duration           0
Stops              0
Price           1486
dtype: int64

In [7]:
df.dropna(inplace=True)

In [8]:
df.shape


(6062, 11)

# Feature Preprocessing on features

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6062 entries, 0 to 7547
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   From          6062 non-null   object        
 1   To            6062 non-null   object        
 2   Booking_date  6062 non-null   datetime64[ns]
 3   Journey_date  6062 non-null   datetime64[ns]
 4   Airline       6062 non-null   object        
 5   Code          6062 non-null   object        
 6   Departure     6062 non-null   object        
 7   Arrival       6062 non-null   object        
 8   Duration      6062 non-null   object        
 9   Stops         6062 non-null   object        
 10  Price         6062 non-null   object        
dtypes: datetime64[ns](2), object(9)
memory usage: 568.3+ KB


In [10]:
df.head(2)

,From,To,Booking_date,Journey_date,Airline,Code,Departure,Arrival,Duration,Stops,Price
0,Delhi,Bangalore,2022-06-23,2022-06-23,Air India,AI-504,21:15,00:05\n+1D,2h 50m,non-stop,"₹9,420"
1,Delhi,Bangalore,2022-06-23,2022-06-23,IndiGo,6E-2131,21:30,00:15\n+1D,2h 45m,non-stop,"₹9,419"


In [11]:
df.groupby(['Airline','Code'])['Code'].count()

Airline   Code   
Air Asia  I5-1529     4
          I5-1732    10
          I5-549      8
          I5-552     14
          I5-710      1
                     ..
Vistara   UK-993     56
          UK-994      6
          UK-995     48
          UK-996      6
          UK-997      1
Name: Code, Length: 632, dtype: int64

In [ ]:
# We are arrangine the flight in between cities

In [14]:
df['Traveling_cities']= df['From']+'-'+df['To']

In [17]:
df['Traveling_cities'].value_counts()

Delhi-Bangalore      1074
Delhi-Mumbai          939
Mumbai-Delhi          885
Bangalore-Delhi       776
Bangalore-kolkata     646
Kolkata-Bangalore     590
Delhi-Goa             588
Goa-Delhi             564
Name: Traveling_cities, dtype: int64

In [19]:
df['Traveling_cities']=df['Traveling_cities'].replace('Bangalore-Delhi','Delhi-Bangalore')
df['Traveling_cities']=df['Traveling_cities'].replace('Mumbai-Delhi','Delhi-Mumbai')
df['Traveling_cities']=df['Traveling_cities'].replace('Kolkata-Bangalore','Bangalore-kolkata')
df['Traveling_cities']=df['Traveling_cities'].replace('Goa-Delhi','Delhi-Goa')


In [20]:
df['Traveling_cities'].value_counts()

Delhi-Bangalore      1850
Delhi-Mumbai         1824
Bangalore-kolkata    1236
Delhi-Goa            1152
Name: Traveling_cities, dtype: int64

In [ ]:
# Create new column with hoe many days early we booked ticket because it has been seen the early you book ticket, the cheaper you get
# Last minute ticket is always expensive

In [24]:
df['Early_booking']=df['Journey_date'].dt.day-  df['Booking_date'].dt.day

In [26]:
df.head(2)

,From,To,Booking_date,Journey_date,Airline,Code,Departure,Arrival,Duration,Stops,Price,Traveling_cities,Early_booking
0,Delhi,Bangalore,2022-06-23,2022-06-23,Air India,AI-504,21:15,00:05\n+1D,2h 50m,non-stop,"₹9,420",Delhi-Bangalore,0
1,Delhi,Bangalore,2022-06-23,2022-06-23,IndiGo,6E-2131,21:30,00:15\n+1D,2h 45m,non-stop,"₹9,419",Delhi-Bangalore,0


In [33]:
df['Airline'].unique()

array(['Air India', 'IndiGo', 'Vistara', 'GO FIRST', 'SpiceJet',
       'Air Asia', 'Indigo', 'AirAsia'], dtype=object)

In [35]:
df['Airline']=df['Airline'].replace('Air Asia','AirAsia')

In [36]:
df['Airline'].unique()

array(['Air India', 'IndiGo', 'Vistara', 'GO FIRST', 'SpiceJet',
       'AirAsia', 'Indigo'], dtype=object)

In [37]:
# Change departure and arrival time in day zone

In [47]:
df['Departure'].dtype

dtype('O')

In [ ]:
# Departure and Arrival is basically the time, lets convert those features in intereg datatype

In [53]:
pd.to_datetime(df['Departure'],format="%H:%M")

0      1900-01-01 21:15:00
1      1900-01-01 21:30:00
2      1900-01-01 22:55:00
3      1900-01-01 21:15:00
4      1900-01-01 21:15:00
               ...        
7543   1900-01-01 11:30:00
7544   1900-01-01 09:45:00
7545   1900-01-01 09:45:00
7546   1900-01-01 11:30:00
7547   1900-01-01 05:10:00
Name: Departure, Length: 6062, dtype: datetime64[ns]

In [56]:
df['Departure'].replace(':','.')

0       21:15
1       21:30
2       22:55
3       21:15
4       21:15
        ...  
7543    11:30
7544    09:45
7545    09:45
7546    11:30
7547    05:10
Name: Departure, Length: 6062, dtype: object

In [ ]:
#bins=[-1,5,11,16,20,24]
#name=['night','Morning','afternoon','evening','midnight']
#df['incident_period_of_the_day']= pd.cut(df['incident_hour_of_the_day'],bins,labels=name)

In [45]:
bins=[-1,5,11,16,19,22,24]
name=['Early_morning_departure','morning_departure','afternoon_departure','evening_departure','night_departure','late_night_departure']

In [46]:
pd.cut(df['Departure'],bins,labels=name)

TypeError: '<' not supported between instances of 'int' and 'str'

In [57]:
df.head()

,From,To,Booking_date,Journey_date,Airline,Code,Departure,Arrival,Duration,Stops,Price,Traveling_cities,Early_booking
0,Delhi,Bangalore,2022-06-23,2022-06-23,Air India,AI-504,21:15,00:05\n+1D,2h 50m,non-stop,"₹9,420",Delhi-Bangalore,0
1,Delhi,Bangalore,2022-06-23,2022-06-23,IndiGo,6E-2131,21:30,00:15\n+1D,2h 45m,non-stop,"₹9,419",Delhi-Bangalore,0
2,Delhi,Bangalore,2022-06-23,2022-06-23,IndiGo,6E-6565,22:55,01:45\n+1D,2h 50m,non-stop,"₹9,419",Delhi-Bangalore,0
3,Delhi,Bangalore,2022-06-23,2022-06-23,Air India,AI-605,21:15,07:50\n+1D,10h 35m,1 stop,"₹14,775",Delhi-Bangalore,0
4,Delhi,Bangalore,2022-06-23,2022-06-23,Air India,AI-605,21:15,18:20\n+1D,21h 5m,1 stop,"₹14,985",Delhi-Bangalore,0


In [60]:
# we have duration in hours and minutes, lets convert complete feature in minutes

df['Duration'].str.split(' ')

0        [2h, 50m]
1        [2h, 45m]
2        [2h, 50m]
3       [10h, 35m]
4        [21h, 5m]
           ...    
7543    [08h, 05m]
7544    [08h, 20m]
7545    [09h, 50m]
7546    [11h, 15m]
7547    [06h, 55m]
Name: Duration, Length: 6062, dtype: object

In [66]:
duration=list(df["Duration"])

for i in range(len(duration)):
    if len(duration[i].split())!=2:
        if "h" in duration[i]:
            duration[i]=duration[i].strip() + " 0m"
        else:
            duration[i]=  "0h "  + duration[i]
            
duration_hours=[]
duration_minutes=[]

for  i in range(len(duration)):
    duration_hours.append(int(duration[i].split("h")[0]))
    duration_minutes.append(int(duration[i].split("m")[0].split()[-1]))
    

df["duration_hours"]=duration_hours
df["duration_minutes"]=duration_minutes

In [67]:
df.head()

,From,To,Booking_date,Journey_date,Airline,Code,Departure,Arrival,Duration,Stops,Price,Traveling_cities,Early_booking,duration_hours,duration_minutes
0,Delhi,Bangalore,2022-06-23,2022-06-23,Air India,AI-504,21:15,00:05\n+1D,2h 50m,non-stop,"₹9,420",Delhi-Bangalore,0,2,50
1,Delhi,Bangalore,2022-06-23,2022-06-23,IndiGo,6E-2131,21:30,00:15\n+1D,2h 45m,non-stop,"₹9,419",Delhi-Bangalore,0,2,45
2,Delhi,Bangalore,2022-06-23,2022-06-23,IndiGo,6E-6565,22:55,01:45\n+1D,2h 50m,non-stop,"₹9,419",Delhi-Bangalore,0,2,50
3,Delhi,Bangalore,2022-06-23,2022-06-23,Air India,AI-605,21:15,07:50\n+1D,10h 35m,1 stop,"₹14,775",Delhi-Bangalore,0,10,35
4,Delhi,Bangalore,2022-06-23,2022-06-23,Air India,AI-605,21:15,18:20\n+1D,21h 5m,1 stop,"₹14,985",Delhi-Bangalore,0,21,5


In [68]:
# Total time travel in hours

df["total_travel_time"]=np.round((df["duration_minutes"]/60)+ df["duration_hours"],2)

In [69]:
df.head()

,From,To,Booking_date,Journey_date,Airline,Code,Departure,Arrival,Duration,Stops,Price,Traveling_cities,Early_booking,duration_hours,duration_minutes,total_travel_time
0,Delhi,Bangalore,2022-06-23,2022-06-23,Air India,AI-504,21:15,00:05\n+1D,2h 50m,non-stop,"₹9,420",Delhi-Bangalore,0,2,50,2.83
1,Delhi,Bangalore,2022-06-23,2022-06-23,IndiGo,6E-2131,21:30,00:15\n+1D,2h 45m,non-stop,"₹9,419",Delhi-Bangalore,0,2,45,2.75
2,Delhi,Bangalore,2022-06-23,2022-06-23,IndiGo,6E-6565,22:55,01:45\n+1D,2h 50m,non-stop,"₹9,419",Delhi-Bangalore,0,2,50,2.83
3,Delhi,Bangalore,2022-06-23,2022-06-23,Air India,AI-605,21:15,07:50\n+1D,10h 35m,1 stop,"₹14,775",Delhi-Bangalore,0,10,35,10.58
4,Delhi,Bangalore,2022-06-23,2022-06-23,Air India,AI-605,21:15,18:20\n+1D,21h 5m,1 stop,"₹14,985",Delhi-Bangalore,0,21,5,21.08


In [70]:
# Here,  Duration feature extracted into duration hour and minute, then we have calculated total travel time in hours

In [73]:
df['Stops'].unique()

array(['non-stop', '1 stop', '2 stops', '1-stop', '2+-stop',
       '1-stop Via Mumbai'], dtype=object)

In [74]:
dic={'non-stop':0,'1 stop':1,'1-stop':1,'1-stop Via Mumbai':1,'2 stops':2,'2+-stop':2}

In [78]:
df['Stops']=df['Stops'].map(dic)

## Dependent feature- Price

In [82]:
df['Price']=df['Price'].str.replace("₹","")
df['Price']=df['Price'].str.replace(",","")
df['Price']=df['Price'].astype(int)